## This Script shows how to use the conversion scripts to convert AMASS npz data into a Unity-Friendly JSON file.

Note that these scripts were written in python 3.8, and therefore probably require at least python 3.6 to run. 
You'll need an up-to-date version of scipy package since older versions do not include the scipy.spatial.transform module. 

Requirements:
* scipy
* numpy
* json

Run This cell to make sure you have dependencies installed.


In [1]:
# Check dependencies
import version_check_util as vers
import scipy
vers.check_version("scipy", scipy.__version__, "1.4.1")
import numpy
vers.check_version("numpy", numpy.__version__, "1.18.1")
import json
vers.check_version("json", json.__version__, "2.0.9")

# Internal Imports
from AmassData import AMASSDataToJSON


scipy version 1.4.1, good
numpy version 1.18.1, good
json version 2.0.9, good


# Convert SMPL-H AMASS animation .npz data files to Unity:


Point to smpl-h npz animation file


In [2]:
npz_data_filename = 'AMASS_Data/punching_poses.npz'


Create converter


In [3]:
converter = AMASSDataToJSON(npz_data_filename)


59.9999885559082
gender: female
betas: (16,)
poses: (234, 156)
dmpls: (234, 8)
trans: (234, 3)
frames detected: 234
fps: 60


Write to output json (full path can be given. this example puts it in the current folder)


In [4]:
converter.write_to_json('test_data.json')


Second data conversion example


In [5]:
sample_data = "AMASS_Data/amass_sample.npz"
converter = AMASSDataToJSON(sample_data)
converter.write_to_json('amass_sample.json')


120.0
gender: male
betas: (16,)
poses: (601, 156)
dmpls: (601, 8)
trans: (601, 3)
frames detected: 601
fps: 120


For batch runs, you can suppress most messages by adding show_messages=False when creating the converter.

In [ ]:
converter = AMASSDataToJSON(npz_data_filename, show_messages=False)

You can also call these scripts from the command line using (assuming you're in the right folder)

`python AmassData.py 'Path/To/npzFile.npz' 'Path/to/destination/jsonFile.json'`